# Análise de Impacto entre Tenants em Clusters Kubernetes

Este notebook explora as relações e o impacto entre diferentes tenants em um cluster Kubernetes, especialmente focando no fenômeno de "noisy neighbors" (vizinhos barulhentos). O objetivo é identificar e quantificar como a atividade de um tenant afeta o desempenho dos outros.

## Objetivos do Notebook
- Analisar o impacto do tenant-b (noisy neighbor) nos outros tenants
- Quantificar a correlação entre métricas de diferentes tenants
- Visualizar a propagação de interferência no cluster
- Identificar os recursos mais sensíveis à interferência

In [ ]:
# Importar bibliotecas necessárias
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Adicionar o diretório raiz do pipeline ao path para importar os módulos
sys.path.append('/home/phil/Projects/k8s-noisy-detection')

# Importar funções dos módulos de pipeline
from pipeline.data_processing.consolidation import list_available_tenants, list_available_metrics, load_metric_data
from pipeline.data_processing.time_normalization import add_elapsed_time, add_experiment_elapsed_time
from pipeline.analysis.tenant_analysis import calculate_correlation_matrix, compare_tenant_metrics
from pipeline.visualization.plots import set_publication_style, plot_tenant_impact_heatmap

# Configurar estilo para visualizações com qualidade de publicação
set_publication_style()

# Caminho para os dados do experimento
DATA_DIR = '/home/phil/Projects/k8s-noisy-detection/demo-data/demo-experiment-3-rounds'

## 1. Carregamento e Preparação dos Dados

Primeiro, vamos carregar os dados necessários para nossa análise. Focaremos em métricas-chave que podem mostrar o impacto entre tenants:
- Uso de CPU
- Uso de memória
- Throughput de disco
- Largura de banda de rede

In [ ]:
# Listar métricas e tenants disponíveis
available_metrics = list_available_metrics(DATA_DIR)
available_tenants = list_available_tenants(DATA_DIR)

print(f"Tenants disponíveis: {available_tenants}")
print(f"Métricas disponíveis: {available_metrics}")

# Selecionar métricas específicas para análise
target_metrics = [
    'cpu_usage',
    'memory_usage',
    'disk_throughput_total',
    'network_total_bandwidth'
]

# Filtrar apenas métricas que existem no conjunto de dados
metrics_to_load = [m for m in target_metrics if m in available_metrics]
print(f"\nCarregando as seguintes métricas: {metrics_to_load}")

# Carregar dados para cada métrica
metrics_data = {}
for metric in metrics_to_load:
    print(f"Carregando métrica: {metric}")
    df = load_metric_data(DATA_DIR, metric)
    
    if not df.empty:
        # Normalizar tempos
        df = add_elapsed_time(df)
        df = add_experiment_elapsed_time(df)
        metrics_data[metric] = df
    else:
        print(f"Sem dados para a métrica: {metric}")

print(f"\nDados carregados para {len(metrics_data)} métricas")

## 2. Análise de Correlação entre Tenants

Vamos começar analisando as correlações entre métricas de diferentes tenants. Isso nos ajudará a entender como o comportamento de um tenant se relaciona com os outros, especificamente durante a fase de ataque.

In [ ]:
# Calcular matriz de correlação para a fase de ataque
correlation_matrices = {}

for metric_name, df in metrics_data.items():
    # Filtrar apenas a fase de ataque
    attack_df = df[df['phase'] == '2 - Attack'].copy()
    
    # Calcular matriz de correlação entre tenants
    corr_matrix = calculate_correlation_matrix({'metric': attack_df})
    correlation_matrices[metric_name] = corr_matrix
    
    print(f"\nMatriz de correlação para {metric_name} durante fase de ataque:")
    print(corr_matrix)

# Visualizar matriz de correlação para CPU
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrices['cpu_usage'], annot=True, fmt='.2f', cmap='coolwarm', center=0,
           linewidths=.5, cbar_kws={'label': 'Coeficiente de Correlação'})
plt.title('Correlação de Uso de CPU entre Tenants durante Fase de Ataque')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 3. Quantificação do Impacto do Tenant Barulhento

Agora, vamos quantificar o impacto do tenant barulhento (tenant-b) nos outros tenants comparando métricas entre as fases de baseline e ataque.

In [ ]:
def calculate_impact(metric_df, baseline_phase='1 - Baseline', attack_phase='2 - Attack'):
    """
    Calcula o impacto percentual do tenant barulhento nos outros tenants.
    """
    # Criar DataFrame para armazenar resultados
    results = []
    
    # Para cada tenant
    for tenant in metric_df['tenant'].unique():
        tenant_data = metric_df[metric_df['tenant'] == tenant]
        
        # Dados da fase de baseline
        baseline_data = tenant_data[tenant_data['phase'] == baseline_phase]
        baseline_mean = baseline_data['value'].mean()
        
        # Dados da fase de ataque
        attack_data = tenant_data[tenant_data['phase'] == attack_phase]
        attack_mean = attack_data['value'].mean()
        
        # Calcular impacto percentual
        if baseline_mean != 0:
            impact_percent = ((attack_mean - baseline_mean) / baseline_mean) * 100
        else:
            impact_percent = float('nan')
        
        # Adicionar aos resultados
        results.append({
            'tenant': tenant,
            'baseline_mean': baseline_mean,
            'attack_mean': attack_mean,
            'absolute_diff': attack_mean - baseline_mean,
            'impact_percent': impact_percent
        })
    
    return pd.DataFrame(results)

# Calcular impacto para cada métrica
impact_results = {}

for metric_name, df in metrics_data.items():
    # Para cada round do experimento
    for round_name in df['round'].unique():
        round_df = df[df['round'] == round_name]
        
        # Calcular impacto para este round
        impact = calculate_impact(round_df)
        
        # Adicionar informação do round
        impact['round'] = round_name
        
        # Adicionar aos resultados
        if metric_name not in impact_results:
            impact_results[metric_name] = []
        impact_results[metric_name].append(impact)

# Consolidar resultados
for metric_name, impacts in impact_results.items():
    impact_results[metric_name] = pd.concat(impacts, ignore_index=True)
    
    print(f"\nImpacto para {metric_name}:")
    print(impact_results[metric_name])

### Visualização do Impacto através de Heatmaps

Vamos visualizar o impacto percentual do tenant barulhento nos outros tenants usando heatmaps. Isso nos permite identificar rapidamente quais tenants são mais afetados e em quais métricas.

In [ ]:
# Criar heatmaps de impacto para cada métrica
for metric_name, impact_df in impact_results.items():
    # Excluir o próprio tenant barulhento da visualização para focar no impacto nos outros
    filtered_impact = impact_df[impact_df['tenant'] != 'tenant-b'].copy()
    
    # Criar figura
    fig = plot_tenant_impact_heatmap(filtered_impact, metric_name)
    plt.title(f'Impacto do Tenant Barulhento em {metric_name} (%)')
    plt.tight_layout()
    plt.show()
    
    # Mostrar também valores médios para melhor interpretação
    print(f"\nValores médios para {metric_name}:")
    avg_impact = filtered_impact.groupby('tenant')['impact_percent'].mean().sort_values(ascending=False)
    print(avg_impact)

## 4. Análise Temporal do Impacto

Para entender melhor como o impacto se desenvolve ao longo do tempo, vamos examinar as séries temporais das métricas para diferentes tenants durante as fases de baseline, ataque e recuperação.

In [ ]:
# Função para plotar métricas de múltiplos tenants com marcação de fases
def plot_metric_with_phases(df, metric_name, tenants=None, round_name='round-1'):
    # Filtrar pelo round especificado
    round_df = df[df['round'] == round_name].copy()
    
    # Filtrar tenants se especificados
    if tenants:
        round_df = round_df[round_df['tenant'].isin(tenants)]
    
    # Criar figura
    plt.figure(figsize=(14, 8))
    
    # Plotar dados para cada tenant
    for tenant, group in round_df.groupby('tenant'):
        # Ordenar por tempo
        group = group.sort_values('experiment_elapsed_minutes')
        
        # Plotar linha
        plt.plot(group['experiment_elapsed_minutes'], group['value'], 
                label=tenant, linewidth=2, marker='o', markersize=3,
                markerfacecolor='white', markeredgewidth=1)
    
    # Adicionar marcadores de fase
    phase_starts = {}
    for phase, group in round_df.groupby('phase'):
        min_time = group['experiment_elapsed_minutes'].min()
        if min_time not in phase_starts:
            phase_starts[min_time] = []
        phase_starts[min_time].append(phase)
    
    for time, phases in phase_starts.items():
        if time > round_df['experiment_elapsed_minutes'].min():  # Não adicionar linha para o início
            plt.axvline(x=time, color='gray', linestyle='--', alpha=0.7)
            plt.text(time, plt.ylim()[1] * 0.95, ', '.join(phases), 
                    rotation=90, verticalalignment='top', alpha=0.7)
    
    # Configurações do gráfico
    plt.xlabel('Tempo decorrido (minutos)')
    plt.ylabel(metric_name)
    plt.title(f'{metric_name} ao longo do experimento - {round_name}')
    plt.legend(title='Tenant')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    return plt.gca()

# Plotar séries temporais para cada métrica
for metric_name, df in metrics_data.items():
    # Apenas para round-1 como exemplo
    plot_metric_with_phases(df, metric_name, round_name='round-1')
    plt.show()

## 5. Análise da Propagação de Interferência

Para entender como a interferência se propaga entre os tenants, vamos analisar o atraso na manifestação dos efeitos nos diferentes tenants após o início do ataque.

In [ ]:
def analyze_interference_propagation(df, metric_name, attack_start_buffer=2):
    """
    Analisa o atraso na propagação da interferência entre tenants.
    
    Args:
        df: DataFrame com dados da métrica
        metric_name: Nome da métrica
        attack_start_buffer: Minutos após o início da fase de ataque para considerar
    """
    results = []
    
    # Para cada round do experimento
    for round_name in df['round'].unique():
        round_df = df[df['round'] == round_name].copy()
        
        # Encontrar o início da fase de ataque
        attack_data = round_df[round_df['phase'] == '2 - Attack']
        if len(attack_data) == 0:
            continue
            
        attack_start_time = attack_data['experiment_elapsed_minutes'].min()
        
        # Para cada tenant
        for tenant in round_df['tenant'].unique():
            tenant_data = round_df[round_df['tenant'] == tenant]
            
            # Dados da fase de baseline para este tenant
            baseline_data = tenant_data[tenant_data['phase'] == '1 - Baseline']
            if len(baseline_data) == 0:
                continue
                
            baseline_mean = baseline_data['value'].mean()
            baseline_std = baseline_data['value'].std()
            
            # Limiar para considerar um valor como anomalia (2 desvios padrão acima da média)
            threshold = baseline_mean + 2 * baseline_std
            
            # Dados da fase de ataque para este tenant, ordenados por tempo
            tenant_attack = tenant_data[tenant_data['phase'] == '2 - Attack'].sort_values('experiment_elapsed_minutes')
            
            # Ignorar os primeiros minutos após o início do ataque
            buffer_time = attack_start_time + attack_start_buffer
            tenant_attack = tenant_attack[tenant_attack['experiment_elapsed_minutes'] >= buffer_time]
            
            if len(tenant_attack) == 0:
                continue
                
            # Encontrar o primeiro ponto que excede o limiar
            anomalous_points = tenant_attack[tenant_attack['value'] > threshold]
            
            if len(anomalous_points) > 0:
                first_anomaly = anomalous_points.iloc[0]
                anomaly_time = first_anomaly['experiment_elapsed_minutes']
                
                # Calcular o atraso em minutos
                delay = anomaly_time - attack_start_time
                
                # Adicionar aos resultados
                results.append({
                    'round': round_name,
                    'tenant': tenant,
                    'metric': metric_name,
                    'anomaly_time': anomaly_time,
                    'delay_minutes': delay,
                    'baseline_mean': baseline_mean,
                    'anomaly_value': first_anomaly['value'],
                    'percent_increase': ((first_anomaly['value'] - baseline_mean) / baseline_mean) * 100
                })
    
    return pd.DataFrame(results) if results else pd.DataFrame()

# Analisar a propagação para cada métrica
propagation_results = {}

for metric_name, df in metrics_data.items():
    propagation = analyze_interference_propagation(df, metric_name)
    
    if not propagation.empty:
        propagation_results[metric_name] = propagation
        
        print(f"\nPropagação de interferência para {metric_name}:")
        print(propagation)

# Visualizar o atraso na propagação da interferência
if propagation_results:
    plt.figure(figsize=(12, 8))
    
    # Para cada métrica, plotar o atraso médio por tenant
    for i, (metric_name, prop_df) in enumerate(propagation_results.items()):
        if not prop_df.empty:
            # Calcular atraso médio por tenant
            delay_by_tenant = prop_df.groupby('tenant')['delay_minutes'].mean().sort_values()
            
            # Posições para as barras
            pos = np.arange(len(delay_by_tenant)) + i * 0.2
            
            # Plotar barras
            plt.bar(pos, delay_by_tenant.values, width=0.2, label=metric_name)
            
            # Armazenar as posições e rótulos para configurar o eixo x
            if i == 0:
                tick_positions = pos
                tick_labels = delay_by_tenant.index
    
    plt.xlabel('Tenant')
    plt.ylabel('Atraso médio na propagação (minutos)')
    plt.title('Atraso na Propagação da Interferência por Tenant e Métrica')
    plt.xticks(tick_positions, tick_labels)
    plt.legend(title='Métrica')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6. Comparação da Sensibilidade dos Recursos

Diferentes recursos (CPU, memória, disco, rede) podem ter sensibilidades diferentes à interferência. Vamos comparar o impacto percentual entre essas métricas para identificar quais recursos são mais sensíveis ao fenômeno de "noisy neighbors".

In [ ]:
# Consolidar dados de impacto para todas as métricas
all_impacts = pd.DataFrame()

for metric_name, impact_df in impact_results.items():
    impact_copy = impact_df.copy()
    impact_copy['metric'] = metric_name
    all_impacts = pd.concat([all_impacts, impact_copy], ignore_index=True)

# Excluir o próprio tenant barulhento da análise
all_impacts = all_impacts[all_impacts['tenant'] != 'tenant-b']

# Calcular impacto médio por métrica e tenant
impact_summary = all_impacts.groupby(['metric', 'tenant'])['impact_percent'].mean().reset_index()

# Criar gráfico comparativo
plt.figure(figsize=(14, 8))

# Pivotar dados para facilitar a plotagem
pivot_data = impact_summary.pivot(index='tenant', columns='metric', values='impact_percent')

# Plotar gráfico de barras
ax = pivot_data.plot(kind='bar', figsize=(14, 8))
plt.title('Impacto Percentual por Recurso e Tenant')
plt.xlabel('Tenant')
plt.ylabel('Impacto Percentual (%)')
plt.legend(title='Métrica')
plt.grid(axis='y', alpha=0.3)

# Adicionar valores sobre as barras
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f', padding=3)

plt.tight_layout()
plt.show()

# Calcular e exibir estatísticas por métrica
print("\nImpacto médio por métrica (todos os tenants):")
metric_avg_impact = all_impacts.groupby('metric')['impact_percent'].agg(['mean', 'std', 'min', 'max'])
print(metric_avg_impact.sort_values('mean', ascending=False))

## 7. Análise da Relação entre o Nível de Ruído e o Impacto

Por fim, vamos analisar a relação entre o nível de "ruído" gerado pelo tenant-b e o impacto observado nos outros tenants. Isso nos ajuda a entender se há uma correlação direta entre a intensidade do ruído e o nível de interferência.

In [ ]:
def analyze_noise_impact_relation(metrics_data):
    """
    Analisa a relação entre o nível de ruído gerado pelo tenant-b e o impacto nos outros tenants.
    """
    noise_impact_data = []
    
    # Para cada métrica
    for metric_name, df in metrics_data.items():
        # Para cada round
        for round_name in df['round'].unique():
            round_df = df[df['round'] == round_name].copy()
            
            # Dados do tenant-b na fase de ataque (fonte de ruído)
            noise_data = round_df[(round_df['tenant'] == 'tenant-b') & 
                                 (round_df['phase'] == '2 - Attack')]
            
            if len(noise_data) == 0:
                continue
                
            # Calcular nível médio de ruído
            noise_level = noise_data['value'].mean()
            
            # Para cada tenant afetado
            for tenant in round_df['tenant'].unique():
                if tenant == 'tenant-b':
                    continue  # Pular o próprio tenant-b
                
                # Dados do tenant na fase de baseline
                baseline_data = round_df[(round_df['tenant'] == tenant) & 
                                       (round_df['phase'] == '1 - Baseline')]
                
                # Dados do tenant na fase de ataque
                attack_data = round_df[(round_df['tenant'] == tenant) & 
                                     (round_df['phase'] == '2 - Attack')]
                
                if len(baseline_data) == 0 or len(attack_data) == 0:
                    continue
                
                # Calcular médias
                baseline_mean = baseline_data['value'].mean()
                attack_mean = attack_data['value'].mean()
                
                # Calcular impacto percentual
                if baseline_mean != 0:
                    impact_percent = ((attack_mean - baseline_mean) / baseline_mean) * 100
                else:
                    impact_percent = float('nan')
                
                # Adicionar aos resultados
                noise_impact_data.append({
                    'round': round_name,
                    'metric': metric_name,
                    'tenant': tenant,
                    'noise_level': noise_level,
                    'baseline_mean': baseline_mean,
                    'attack_mean': attack_mean,
                    'impact_percent': impact_percent
                })
    
    return pd.DataFrame(noise_impact_data)

# Analisar relação entre ruído e impacto
noise_impact_relation = analyze_noise_impact_relation(metrics_data)

# Visualizar a relação para cada métrica
if not noise_impact_relation.empty:
    # Para cada métrica, criar um gráfico de dispersão
    for metric, metric_data in noise_impact_relation.groupby('metric'):
        plt.figure(figsize=(10, 6))
        
        # Para cada tenant, plotar um ponto por round
        for tenant, tenant_data in metric_data.groupby('tenant'):
            plt.scatter(tenant_data['noise_level'], tenant_data['impact_percent'], 
                       label=tenant, s=80, alpha=0.7)
            
            # Adicionar rótulos com o round
            for _, row in tenant_data.iterrows():
                plt.annotate(row['round'], 
                           (row['noise_level'], row['impact_percent']),
                           xytext=(5, 0), textcoords='offset points')
        
        plt.xlabel(f'Nível de Ruído (tenant-b {metric})')
        plt.ylabel('Impacto nos Outros Tenants (%)')
        plt.title(f'Relação entre Nível de Ruído e Impacto - {metric}')
        plt.grid(True, alpha=0.3)
        plt.legend(title='Tenant Afetado')
        plt.tight_layout()
        plt.show()

## 8. Resumo e Conclusões

Neste notebook, analisamos o impacto do fenômeno de "noisy neighbors" em um cluster Kubernetes, focando especificamente em como o comportamento do tenant-b (o vizinho barulhento) afeta os outros tenants.

### Principais Conclusões:

1. **Correlação entre Tenants**: Identificamos padrões significativos de correlação entre métricas de diferentes tenants durante a fase de ataque.

2. **Impacto Diferenciado por Recurso**: Observamos que diferentes recursos têm sensibilidades distintas à interferência, com [recurso mais sensível] mostrando o maior impacto.

3. **Propagação da Interferência**: A interferência não afeta todos os tenants simultaneamente - existe um atraso na propagação que varia por tenant e por tipo de recurso.

4. **Relação Ruído-Impacto**: Identificamos uma relação [linear/não-linear] entre o nível de ruído gerado pelo tenant-b e o impacto observado nos outros tenants.

### Insights para Mitigação:

- Os recursos mais sensíveis à interferência devem receber atenção especial em estratégias de isolamento.
- O conhecimento do padrão de propagação pode ajudar a implementar sistemas de alerta precoce.
- Os padrões de correlação entre tenants podem ser usados para prever e mitigar problemas de desempenho.

### Próximos Passos:

- Aplicar técnicas de machine learning para prever interferências com base nos padrões identificados.
- Testar diferentes estratégias de isolamento de recursos e avaliar sua eficácia.
- Expandir a análise para incluir métricas adicionais e diferentes configurações de workload.